In [35]:
import pandas as pd
import numpy as np
import random

In [36]:
# Prepare the data and depot location
customers_df = pd.read_csv("customers.csv")
vehicles_df = pd.read_csv("vehicles.csv")

depot = (4.4184, 114.0932)

In [37]:
customers_df.head()

,customer,latitude,longitude,demand
0,1,4.3555,113.9777,5
1,2,4.3976,114.0049,8
2,3,4.3163,114.0764,3
3,4,4.3184,113.9932,6
4,5,4.4024,113.9896,5


In [38]:
vehicles_df.head()

,vehicle,capacity,cost_per_km
0,Type A,25,1.2
1,Type B,30,1.5


In [39]:
# Convert the DataFrames to lists of dictionaries
customers = customers_df.to_dict('records')
vehicles_data = vehicles_df.to_dict('records')

# Print the converted data to verify
print("Customers (converted):")
print(customers)

print("\nVehicles (converted):")
print(vehicles_data)


Customers (converted):
[{'customer': 1, 'latitude': 4.3555, 'longitude': 113.9777, 'demand': 5}, {'customer': 2, 'latitude': 4.3976, 'longitude': 114.0049, 'demand': 8}, {'customer': 3, 'latitude': 4.3163, 'longitude': 114.0764, 'demand': 3}, {'customer': 4, 'latitude': 4.3184, 'longitude': 113.9932, 'demand': 6}, {'customer': 5, 'latitude': 4.4024, 'longitude': 113.9896, 'demand': 5}, {'customer': 6, 'latitude': 4.4142, 'longitude': 114.0127, 'demand': 8}, {'customer': 7, 'latitude': 4.0804, 'longitude': 114.0734, 'demand': 3}, {'customer': 8, 'latitude': 4.3818, 'longitude': 114.2034, 'demand': 6}, {'customer': 9, 'latitude': 4.4935, 'longitude': 114.1828, 'demand': 5}, {'customer': 10, 'latitude': 4.4932, 'longitude': 114.1322, 'demand': 8}]

Vehicles (converted):
[{'vehicle': 'Type A', 'capacity': 25, 'cost_per_km': 1.2}, {'vehicle': 'Type B', 'capacity': 30, 'cost_per_km': 1.5}]


In [40]:
# Helper function to calculate the Euclidean distance between two points
def calculate_distance(point1, point2):
    """
    Calculate the Euclidean distance between two points.
    
    Args:
    point1, point2: Tuples representing the coordinates (latitude, longitude) of the two points.
    
    Returns:
    A float representing the distance between the two points.
    """
    return 100 * np.sqrt((point2[1] - point1[1])**2 + (point2[0] - point1[0])**2)

# Example usage
point1 = (4.4184, 114.0932)
point2 = (4.4201, 114.1000)
distance = calculate_distance(point1, point2)
print(f"Distance between {point1} and {point2} is {distance:.3f} km")


Distance between (4.4184, 114.0932) and (4.4201, 114.1) is 0.701 km


In [41]:
def generate_initial_population(pop_size, customers, vehicles_data):
    """
    Generate an initial population of solutions for the VRP.
    
    Args:
    pop_size: The number of solutions in the population.
    customers: A list of dictionaries containing customer data.
    vehicles_data: A list of dictionaries containing vehicle data.
    
    Returns:
    A list of solutions, where each solution is a list of routes for the vehicles.
    """
    population = []
    for _ in range(pop_size):
        solution = []
        remaining_customers = customers.copy()
        for vehicle in vehicles_data:
            route = []
            route_demand = 0
            while remaining_customers and (route_demand + remaining_customers[0]['demand']) <= vehicle['capacity']:
                customer = remaining_customers.pop(random.randint(0, len(remaining_customers) - 1))
                route.append(customer)
                route_demand += customer['demand']
            solution.append({'Vehicle': vehicle['vehicle'], 'route': route})  # Use 'vehicle' instead of 'type'
        population.append(solution)
    return population

# Example usage
population_size = 10
initial_population = generate_initial_population(population_size, customers, vehicles_data)
print("Initial Population:")
for i, solution in enumerate(initial_population):
    print(f"Solution {i+1}: {solution}")

Initial Population:
Solution 1: [{'Vehicle': 'Type A', 'route': [{'customer': 2, 'latitude': 4.3976, 'longitude': 114.0049, 'demand': 8}, {'customer': 3, 'latitude': 4.3163, 'longitude': 114.0764, 'demand': 3}, {'customer': 8, 'latitude': 4.3818, 'longitude': 114.2034, 'demand': 6}, {'customer': 1, 'latitude': 4.3555, 'longitude': 113.9777, 'demand': 5}]}, {'Vehicle': 'Type B', 'route': [{'customer': 5, 'latitude': 4.4024, 'longitude': 113.9896, 'demand': 5}, {'customer': 9, 'latitude': 4.4935, 'longitude': 114.1828, 'demand': 5}, {'customer': 4, 'latitude': 4.3184, 'longitude': 113.9932, 'demand': 6}, {'customer': 10, 'latitude': 4.4932, 'longitude': 114.1322, 'demand': 8}]}]
Solution 2: [{'Vehicle': 'Type A', 'route': [{'customer': 8, 'latitude': 4.3818, 'longitude': 114.2034, 'demand': 6}, {'customer': 5, 'latitude': 4.4024, 'longitude': 113.9896, 'demand': 5}, {'customer': 6, 'latitude': 4.4142, 'longitude': 114.0127, 'demand': 8}, {'customer': 1, 'latitude': 4.3555, 'longitude': 1

In [42]:
def evaluate(solution):
    """
    Evaluate the fitness of a solution.
    
    Args:
    solution: A solution representing a set of routes for the vehicles.
    
    Returns:
    A float representing the total cost of the solution. If the solution is infeasible, returns a high penalty value.
    """
    total_distance = 0
    total_cost = 0
    
    for vehicle in solution:
        vehicle_type = vehicle['Vehicle']
        vehicle_capacity = vehicles_df.loc[vehicles_df['vehicle'] == vehicle_type, 'capacity'].values[0]
        cost_per_km = vehicles_df.loc[vehicles_df['vehicle'] == vehicle_type, 'cost_per_km'].values[0]
        
        route = vehicle['route']
        route_demand = sum(customer['demand'] for customer in route)
        
        if route_demand > vehicle_capacity:
            return float('inf')  # Penalize infeasible solutions
        
        route_distance = 0
        last_location = depot
        for customer in route:
            customer_location = (customer['latitude'], customer['longitude'])
            route_distance += calculate_distance(last_location, customer_location)
            last_location = customer_location
        route_distance += calculate_distance(last_location, depot)  # Return to depot
        total_distance += route_distance
        total_cost += route_distance * cost_per_km
    
    return total_cost

# Example usage
example_solution = initial_population[0]
fitness = evaluate(example_solution)
print(f"Fitness of the example solution: {fitness}")

Fitness of the example solution: 216.70509954886586


In [43]:
def selection(population, fitnesses):
    """
    Select solutions from the population to act as parents for the next generation.
    
    Args:
    population: The current population of solutions.
    fitnesses: A list of fitness values corresponding to each solution in the population.
    
    Returns:
    A list of selected solutions (parents).
    """
    total_fitness = sum(1 / f for f in fitnesses)  # Inverse fitness for minimization problem
    selection_probs = [(1 / f) / total_fitness for f in fitnesses]
    
    selected = random.choices(population, weights=selection_probs, k=len(population))
    return selected

# Example usage
fitnesses = [evaluate(solution) for solution in initial_population]
selected_population = selection(initial_population, fitnesses)
print("Selected Population:")
for i, solution in enumerate(selected_population):
    print(f"Selected Solution {i+1}: {solution}")

Selected Population:
Selected Solution 1: [{'Vehicle': 'Type A', 'route': [{'customer': 7, 'latitude': 4.0804, 'longitude': 114.0734, 'demand': 3}, {'customer': 3, 'latitude': 4.3163, 'longitude': 114.0764, 'demand': 3}, {'customer': 4, 'latitude': 4.3184, 'longitude': 113.9932, 'demand': 6}, {'customer': 1, 'latitude': 4.3555, 'longitude': 113.9777, 'demand': 5}, {'customer': 2, 'latitude': 4.3976, 'longitude': 114.0049, 'demand': 8}]}, {'Vehicle': 'Type B', 'route': [{'customer': 10, 'latitude': 4.4932, 'longitude': 114.1322, 'demand': 8}, {'customer': 6, 'latitude': 4.4142, 'longitude': 114.0127, 'demand': 8}, {'customer': 8, 'latitude': 4.3818, 'longitude': 114.2034, 'demand': 6}, {'customer': 5, 'latitude': 4.4024, 'longitude': 113.9896, 'demand': 5}]}]
Selected Solution 2: [{'Vehicle': 'Type A', 'route': [{'customer': 2, 'latitude': 4.3976, 'longitude': 114.0049, 'demand': 8}, {'customer': 3, 'latitude': 4.3163, 'longitude': 114.0764, 'demand': 3}, {'customer': 8, 'latitude': 4.3

In [44]:
def crossover(parent1, parent2):
    """
    Perform crossover between two parent solutions to produce two offspring solutions.
    
    Args:
    parent1: The first parent solution.
    parent2: The second parent solution.
    
    Returns:
    Two offspring solutions generated by combining parts of the parents.
    """
    def get_vehicle_routes(solution):
        routes = []
        for vehicle in solution:
            routes.extend(vehicle['route'])
        return routes

    def create_offspring(parent1, parent2):
        route1 = get_vehicle_routes(parent1)
        route2 = get_vehicle_routes(parent2)
        
        size = len(route1)
        start, end = sorted(random.sample(range(size), 2))
        
        child_route = [None] * size
        child_route[start:end] = route1[start:end]
        
        pointer = 0
        for customer in route2:
            if customer not in child_route:
                while pointer < size and child_route[pointer] is not None:
                    pointer += 1
                if pointer < size:
                    child_route[pointer] = customer
        
        return child_route

    def distribute_customers(route, vehicles_data):
        solution = []
        remaining_customers = [c for c in route if c is not None]
        for vehicle in vehicles_data:
            route_segment = []
            route_demand = 0
            while remaining_customers and (route_demand + remaining_customers[0]['demand']) <= vehicle['capacity']:
                customer = remaining_customers.pop(0)
                route_segment.append(customer)
                route_demand += customer['demand']
            solution.append({'Vehicle': vehicle['vehicle'], 'route': route_segment})
        return solution
    
    child1_route = create_offspring(parent1, parent2)
    child2_route = create_offspring(parent2, parent1)
    
    child1 = distribute_customers(child1_route, vehicles_data)
    child2 = distribute_customers(child2_route, vehicles_data)
    
    return child1, child2

# Example usage
parent1 = initial_population[0]
parent2 = initial_population[1]
child1, child2 = crossover(parent1, parent2)
print("Child 1:", child1)
print("Child 2:", child2)


Child 1: [{'Vehicle': 'Type A', 'route': [{'customer': 5, 'latitude': 4.4024, 'longitude': 113.9896, 'demand': 5}, {'customer': 6, 'latitude': 4.4142, 'longitude': 114.0127, 'demand': 8}, {'customer': 8, 'latitude': 4.3818, 'longitude': 114.2034, 'demand': 6}, {'customer': 1, 'latitude': 4.3555, 'longitude': 113.9777, 'demand': 5}]}, {'Vehicle': 'Type B', 'route': [{'customer': 7, 'latitude': 4.0804, 'longitude': 114.0734, 'demand': 3}, {'customer': 3, 'latitude': 4.3163, 'longitude': 114.0764, 'demand': 3}, {'customer': 10, 'latitude': 4.4932, 'longitude': 114.1322, 'demand': 8}, {'customer': 2, 'latitude': 4.3976, 'longitude': 114.0049, 'demand': 8}]}]
Child 2: [{'Vehicle': 'Type A', 'route': [{'customer': 8, 'latitude': 4.3818, 'longitude': 114.2034, 'demand': 6}, {'customer': 1, 'latitude': 4.3555, 'longitude': 113.9777, 'demand': 5}, {'customer': 5, 'latitude': 4.4024, 'longitude': 113.9896, 'demand': 5}, {'customer': 9, 'latitude': 4.4935, 'longitude': 114.1828, 'demand': 5}, {'c

In [45]:
def mutate(solution, mutation_rate=0.1):
    """
    Mutate a solution by introducing random changes.
    
    Args:
    solution: A solution representing a set of routes for the vehicles.
    mutation_rate: The probability of mutation for each customer in a route.
    
    Returns:
    A mutated solution.
    """
    for vehicle in solution:
        route = vehicle['route']
        if random.random() < mutation_rate:
            # Ensure there are at least two customers to swap
            if len(route) > 1:
                i, j = random.sample(range(len(route)), 2)
                route[i], route[j] = route[j], route[i]
    
    return solution

# Example usage
mutated_solution = mutate(child1)
print("Mutated Solution:", mutated_solution)

Mutated Solution: [{'Vehicle': 'Type A', 'route': [{'customer': 5, 'latitude': 4.4024, 'longitude': 113.9896, 'demand': 5}, {'customer': 6, 'latitude': 4.4142, 'longitude': 114.0127, 'demand': 8}, {'customer': 8, 'latitude': 4.3818, 'longitude': 114.2034, 'demand': 6}, {'customer': 1, 'latitude': 4.3555, 'longitude': 113.9777, 'demand': 5}]}, {'Vehicle': 'Type B', 'route': [{'customer': 7, 'latitude': 4.0804, 'longitude': 114.0734, 'demand': 3}, {'customer': 3, 'latitude': 4.3163, 'longitude': 114.0764, 'demand': 3}, {'customer': 10, 'latitude': 4.4932, 'longitude': 114.1322, 'demand': 8}, {'customer': 2, 'latitude': 4.3976, 'longitude': 114.0049, 'demand': 8}]}]


In [46]:
def genetic_algorithm(customers, vehicles_data, depot, pop_size=100, generations=500, mutation_rate=0.1):
    """
    Run the Genetic Algorithm to solve the VRP.
    
    Args:
    customers: A list of dictionaries containing customer data.
    vehicles_data: A list of dictionaries containing vehicle data.
    depot: Tuple representing the coordinates of the depot.
    pop_size: The number of solutions in the population.
    generations: The number of generations to run the algorithm.
    mutation_rate: The probability of mutation for each customer in a route.
    
    Returns:
    The best solution found and its fitness value.
    """
    # Generate initial population
    population = generate_initial_population(pop_size, customers, vehicles_data)
    
    for generation in range(generations):
        # Evaluate fitness of each solution
        fitnesses = [evaluate(solution) for solution in population]
        
        # Selection
        selected_population = selection(population, fitnesses)
        
        # Crossover and Mutation
        new_population = []
        for i in range(0, len(selected_population), 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i+1] if i+1 < len(selected_population) else selected_population[0]
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1, mutation_rate))
            new_population.append(mutate(child2, mutation_rate))
        
        # Replace old population with new population
        population = new_population
    
    # Find the best solution in the final population
    best_solution = min(population, key=evaluate)
    best_fitness = evaluate(best_solution)
    return best_solution, best_fitness

# Run the Genetic Algorithm
best_solution, best_fitness = genetic_algorithm(customers, vehicles_data, depot)
print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)

Best Solution: [{'Vehicle': 'Type A', 'route': [{'customer': 1, 'latitude': 4.3555, 'longitude': 113.9777, 'demand': 5}, {'customer': 4, 'latitude': 4.3184, 'longitude': 113.9932, 'demand': 6}, {'customer': 2, 'latitude': 4.3976, 'longitude': 114.0049, 'demand': 8}]}, {'Vehicle': 'Type B', 'route': [{'customer': 10, 'latitude': 4.4932, 'longitude': 114.1322, 'demand': 8}, {'customer': 9, 'latitude': 4.4935, 'longitude': 114.1828, 'demand': 5}, {'customer': 8, 'latitude': 4.3818, 'longitude': 114.2034, 'demand': 6}, {'customer': 3, 'latitude': 4.3163, 'longitude': 114.0764, 'demand': 3}]}]
Best Fitness: 115.33660310363717
